In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import numpy as np
import random as rn
import tensorflow as tf
def reset_seeds():
    np.random.seed(1) 
    rn.seed(1)
    tf.random.set_seed(1)
reset_seeds() 
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.stem import SnowballStemmer  
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.python.keras.backend import get_session
print(tf.__version__)
from tensorflow.python import keras
print(keras.__version__)
import random as rn
from tensorflow.python.keras.backend import get_session
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Flatten, Dropout, Activation, Input
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.convolutional import MaxPooling1D


2.9.2
2.6.0


In [3]:
from google.colab import files
upload= files.upload()
GOFdataset=pd.read_excel('Douglass.xlsx',index_col=None)
pd.set_option('display.max_colwidth',None)
GOFdataset.head()
GOFdataset['description1']=GOFdataset['description'].str.replace("\r","")
GOFdataset['description1']=GOFdataset['description1'].str.lower()
punc=list("?!^.,\')•(:-][—+;\"//")
GOFdataset['description2']=GOFdataset['description1']
for pun in punc:
    GOFdataset['description2']=GOFdataset['description2'].str.replace(pun,' ')


Saving Douglass.xlsx to Douglass.xlsx


<ipython-input-3-fcee66840e9d>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  GOFdataset['description2']=GOFdataset['description2'].str.replace(pun,' ')


In [4]:
GOFdataset['description3']=GOFdataset['description2'].str.replace("'s", " ")
GOFdataset['description4']=GOFdataset['description3'].str.replace(r'\s+[a-z0-9]\s+', ' ')
#GOFdataset
import nltk
nltk.download('stopwords')
nrow=len (GOFdataset)
tokenizedlist=[]
for row in range(0, nrow):
    lematizedList=[]
    text=GOFdataset.loc[row]['description4']
    text_words=[word for word in text.split() if word not in (stopwords.words('english'))]
    #print(text_words)
    for word in text_words:
        stemmer=SnowballStemmer('english')
        lematizedList.append(stemmer.stem(word))
    stemmed=(lematizedList)
    #print(stemmed)
    tokenizedlist.append(stemmed)
    #print(tokenizedlist) 
GOFdataset['description5']=tokenizedlist
pd.set_option('display.max_colwidth',None)
#print(GOFdataset['description5'])

finalVocab=GOFdataset['description5'].apply(pd.Series).stack().unique().tolist()
print(len(finalVocab))

<ipython-input-4-978e2da986f6>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  GOFdataset['description4']=GOFdataset['description3'].str.replace(r'\s+[a-z0-9]\s+', ' ')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


642


In [5]:
import math
nrow2=len(GOFdataset)
TFvec=[]
for row2 in range(0, nrow2):
    TF={}
    docs=GOFdataset.loc[row2]['description5']
    for words in finalVocab:
        TF[words]=0
    for words in docs:
        TF[words]+=1
        #print(TF)
    TFvec.append(TF)
    #print(TFvec)
GOFdataset['description6']=TFvec
#print(GOFdataset['description6'])
nrow4=len(GOFdataset)
TFVector=[]
for row in range(0, nrow4):
    tee=GOFdataset.loc[row]['description6']
    TFvec=[0]*len(tee)
    for i, words in enumerate(tee):
        TFvec[i]=tee[words]
    #print(TFvec)
    TFVector.append(TFvec)
    #print(TFVector)
pd.set_option('display.max_colwidth',None)
GOFdataset['description7']=TFVector
#GOFdataset['description7']

In [6]:
import numpy as np
from numpy import unique
np.set_printoptions(threshold=np.inf)
TF=np.array([i for i in GOFdataset['description7']]).astype(np.float32)
x = TF.reshape(TF.shape[0], TF.shape[1],1)
print(x.shape)
y=np.array([i for i in GOFdataset['label']])
   

(34, 642, 1)


In [47]:
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.backend import get_session
from tensorflow.python.keras.models import Sequential ,load_model
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Flatten, Dropout, Activation, Input
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.convolutional import MaxPooling1D
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.python.keras import regularizers
from keras import metrics
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler
reset_seeds() 
model = Sequential()
init=keras.initializers.HeNormal(seed=None)
bias=keras.initializers.Constant()
model.add(Conv1D(filters=128, kernel_size=3, padding='same', kernel_initializer=keras.initializers.HeNormal(),input_shape=(642,1)))
model.add(Activation('relu'))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', kernel_initializer=keras.initializers.HeNormal()))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2, data_format='channels_last'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(642,  bias_initializer=bias,name='Dense_2'))
model.add(Activation('relu'))
reset_seeds() 
feature_vectors_model = Model(inputs=model.input, outputs=model.get_layer('Dense_2').output)
dtc_features = feature_vectors_model.predict(x)
scaler = MinMaxScaler()
scaled= scaler.fit_transform(dtc_features)
scores = []
best_svr = MultinomialNB()
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(scaled,y):
    X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.30,  stratify=y)
    best_svr.fit(X_train, y_train)
    pred=(best_svr.predict(X_test))
    scores.append(f1_score(y_test, pred, average='micro'))
print(np.mean(scores))
print((scores))


0.8272727272727272
[0.9090909090909091, 0.9090909090909091, 0.7272727272727273, 0.7272727272727273, 0.9090909090909091, 0.8181818181818182, 0.9090909090909091, 0.8181818181818182, 0.9090909090909091, 0.6363636363636364]
